In [1]:
import netCDF4 as nc
import pandas as pd
import os
import numpy as np

In [3]:
# Open the NetCDF file
dataset = nc.Dataset('data/chirps-v2.0.annual.nc')

# Extract variable
precip = dataset.variables['precip']

# Get dimensions assuming 3D: time, latitude, longitude
time_dim, lat_dim, lon_dim = precip.get_dims()
time_var = dataset.variables[time_dim.name]
times = nc.num2date(time_var[:], time_var.units)
latitudes = dataset.variables[lat_dim.name][:]
longitudes = dataset.variables[lon_dim.name][:]

# Define Bangladesh's geographical boundaries
lat_min, lat_max = 20.5, 26.5
lon_min, lon_max = 88.0, 92.5

# Create a mask for latitudes and longitudes
lat_mask = (latitudes >= lat_min) & (latitudes <= lat_max)
lon_mask = (longitudes >= lon_min) & (longitudes <= lon_max)

# Get indices for the filtered latitudes and longitudes
lat_indices = np.where(lat_mask)[0]
lon_indices = np.where(lon_mask)[0]

# Initialize lists for storing filtered data
filtered_times = []
filtered_latitudes = []
filtered_longitudes = []
filtered_precip = []

# Filter data
for t in range(len(times)):
    for lat_index in lat_indices:
        for lon_index in lon_indices:
            filtered_times.append(times[t].isoformat())
            filtered_latitudes.append(latitudes[lat_index])
            filtered_longitudes.append(longitudes[lon_index])
            filtered_precip.append(precip[t, lat_index, lon_index])

# Create DataFrame
df = pd.DataFrame({
    'time': filtered_times,
    'latitude': filtered_latitudes,
    'longitude': filtered_longitudes,
    'precip': filtered_precip
})

print(df)

# Define output directory and filename
output_dir = 'data/'
filename = os.path.join(output_dir, 'chirps_bangladesh.csv')

# Export to CSV
print(f'Writing data in tabular form to {filename} (this may take some time)...')
df.to_csv(filename, index=False)
print('Done')


KeyboardInterrupt: 

In [12]:
file_path = 'data/journal.pone.0255519.s001.csv'

df = pd.read_csv(file_path)

bangladesh_df = df[df['country'] == 'Bangladesh'].copy()

In [18]:
import geopandas as gpd
from shapely.geometry import Point

# Load geographical regions from a GeoJSON file
regions = gpd.read_file('data/geoBoundaries-BGD-ADM2.geojson')

# Create Points for each latitude and longitude
bangladesh_df['geometry'] = bangladesh_df.apply(lambda row: Point(row['longnum'], row['latnum']), axis=1)

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(bangladesh_df, geometry='geometry')

# Check which region each point falls into and create a new column for regions
gdf['region'] = gdf.apply(
    lambda row: regions[regions.contains(row['geometry'])].iloc[0]['shapeName'] 
    if not regions[regions.contains(row['geometry'])].empty else None, 
    axis=1
)

gdf.to_csv('data/bangladesh_geodata.csv', index=False)

   shapeName shapeISO                  shapeID shapeGroup shapeType  \
0   Bagerhat           16705992B77600989823530        BGD      ADM2   
1  Bandarban           16705992B23463827258150        BGD      ADM2   
2    Barguna           16705992B13795829103768        BGD      ADM2   
3    Barisal           16705992B66943755852345        BGD      ADM2   
4      Bhola           16705992B37732832620169        BGD      ADM2   

                                            geometry  
0  MULTIPOLYGON (((89.84021 21.82784, 89.84011 21...  
1  POLYGON ((92.61391 21.85191, 92.59671 21.86793...  
2  MULTIPOLYGON (((89.87968 22.07869, 89.88024 22...  
3  MULTIPOLYGON (((90.55435 22.59264, 90.55429 22...  
4  MULTIPOLYGON (((90.77172 21.85595, 90.77446 21...  
   Unnamed: 0  URBAN_RURA      alt     chrps     country  deathcount  \
0           0           1  5.60252  0.437479  Bangladesh           0   
1           1           1  4.67774  0.447809  Bangladesh           0   
2           2           1  

In [21]:
# Load the CSV file
df = pd.read_csv('data/bangladesh_geodata.csv')

# Drop specific columns
columns_to_drop = [
    'marketm0', 'marketm1', 'marketm10', 'marketm11', 'marketm12', 
    'marketm13', 'marketm14', 'marketm15', 'marketm16', 'marketm17', 
    'marketm18', 'marketm19', 'marketm2', 'marketm20', 'marketm21', 
    'marketm22', 'marketm23', 'marketm24', 'marketm25', 'marketm26', 
    'marketm27', 'marketm28', 'marketm29', 'marketm3', 'marketm30', 
    'marketm31', 'marketm32', 'marketm33', 'marketm34', 'marketm35', 
    'marketm36', 'marketm37', 'marketm38', 'marketm39', 'marketm4', 
    'marketm40', 'marketm41', 'marketm42', 'marketm43', 'marketm44', 
    'marketm45', 'marketm46', 'marketm47', 'marketm5', 'marketm6', 
    'marketm7', 'marketm8', 'marketm9', 'markets0', 'markets1', 
    'markets10', 'markets11', 'markets12', 'markets13', 'markets14', 
    'markets15', 'markets16', 'markets17', 'markets18', 'markets19', 
    'markets2', 'markets20', 'markets21', 'markets22', 'markets23', 
    'markets24', 'markets25', 'markets26', 'markets27', 'markets28', 
    'markets29', 'markets3', 'markets30', 'markets31', 'markets32', 
    'markets33', 'markets34', 'markets35', 'markets36', 'markets37', 
    'markets38', 'markets39', 'markets4', 'markets40', 'markets41', 
    'markets42', 'markets43', 'markets44', 'markets45', 'markets46', 
    'markets47', 'markets5', 'markets6', 'markets7', 'markets8', 
    'markets9']
df_dropped = df.drop(columns=columns_to_drop)

# Optionally, save the modified DataFrame back to a CSV
df_dropped.to_csv('data/bangladesh_geodata.csv', index=False)

KeyError: "['marketm0', 'marketm1', 'marketm10', 'marketm11', 'marketm12', 'marketm13', 'marketm14', 'marketm15', 'marketm16', 'marketm17', 'marketm18', 'marketm19', 'marketm2', 'marketm20', 'marketm21', 'marketm22', 'marketm23', 'marketm24', 'marketm25', 'marketm26', 'marketm27', 'marketm28', 'marketm29', 'marketm3', 'marketm30', 'marketm31', 'marketm32', 'marketm33', 'marketm34', 'marketm35', 'marketm36', 'marketm37', 'marketm38', 'marketm39', 'marketm4', 'marketm40', 'marketm41', 'marketm42', 'marketm43', 'marketm44', 'marketm45', 'marketm46', 'marketm47', 'marketm5', 'marketm6', 'marketm7', 'marketm8', 'marketm9', 'markets0', 'markets1', 'markets10', 'markets11', 'markets12', 'markets13', 'markets14', 'markets15', 'markets16', 'markets17', 'markets18', 'markets19', 'markets2', 'markets20', 'markets21', 'markets22', 'markets23', 'markets24', 'markets25', 'markets26', 'markets27', 'markets28', 'markets29', 'markets3', 'markets30', 'markets31', 'markets32', 'markets33', 'markets34', '